In [1]:
import requests
import json 
from time import sleep, time
from pandas import json_normalize
import pandas as pd

with open("token.txt", "r") as file:
    token = file.read().strip('\n')

base_url = "https://api.clashroyale.com/v1"

headers = {"Authorization": f"Bearer {token}"}

# Archivo donde guardaremos los resultados
output_file = "clanes_data.json"

# Lista de tags de clanes (ejemplo)
clan_tags = ["#L9CRC0PG", "#GG0C0VR0", "#P00CUJ2R", "#LC0GJGRV", "#YVQV2VVY", "#9JUP2U", "#RPQ2QGYC"] 

def get_clan_data(clan_tag):
    endpoint = f"/clans/{clan_tag.replace('#', '%23')}"  # Codificar el # como %23
    url = base_url + endpoint
    
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        sleep(0.1)  # Pequeña pausa para evitar rate limiting
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Error al obtener datos del clan {clan_tag}: {e}")
        return None

def fetch_all_clans_data(tags, output_file):
    start_time = time()  # Iniciar medición de tiempo
    all_data = {}
    total_clans = len(tags)
    
    for i, tag in enumerate(tags, 1):
        print(f"Obteniendo datos del clan {tag} ({i}/{total_clans})...")
        clan_data = get_clan_data(tag)
        
        if clan_data is not None:
            all_data[tag] = clan_data
    
    # Guardar todos los datos
    with open(output_file, "w") as f:
        json.dump(all_data, f, indent=4)
    
    # Calcular tiempo transcurrido
    elapsed_time = time() - start_time
    mins, secs = divmod(elapsed_time, 60)
    
    print(f"\nProceso completado en {int(mins)} minutos y {secs:.2f} segundos")
    print(f"Datos de {len(all_data)}/{total_clans} clanes guardados en {output_file}")
    
    return all_data

#nota: Se tiene que revisar la IP del Codespace

In [2]:
fetch_all_clans_data(clan_tags, output_file)

Obteniendo datos del clan #L9CRC0PG (1/7)...
Obteniendo datos del clan #GG0C0VR0 (2/7)...
Obteniendo datos del clan #P00CUJ2R (3/7)...
Obteniendo datos del clan #LC0GJGRV (4/7)...
Obteniendo datos del clan #YVQV2VVY (5/7)...
Obteniendo datos del clan #9JUP2U (6/7)...
Obteniendo datos del clan #RPQ2QGYC (7/7)...

Proceso completado en 0 minutos y 1.85 segundos
Datos de 7/7 clanes guardados en clanes_data.json


{'#L9CRC0PG': {'tag': '#L9CRC0PG',
  'name': 'Biscy Saucers',
  'type': 'inviteOnly',
  'description': '1# Clan in Canada 🇨🇦 Better than the Rest! Very Active and Supportive! Path of Legends Focused 🏆 Full? Join biscy saucers 2!',
  'badgeId': 16000076,
  'clanScore': 90000,
  'clanWarTrophies': 1650,
  'location': {'id': 57000047,
   'name': 'Canada',
   'isCountry': True,
   'countryCode': 'CA'},
  'requiredTrophies': 9000,
  'donationsPerWeek': 2760,
  'clanChestStatus': 'inactive',
  'clanChestLevel': 1,
  'clanChestMaxLevel': 0,
  'members': 50,
  'memberList': [{'tag': '#JJU8V9YR',
    'name': 'BigDanny224',
    'role': 'coLeader',
    'lastSeen': '20250530T031133.000Z',
    'expLevel': 69,
    'trophies': 9000,
    'arena': {'id': 54000020, 'name': 'Valkalla'},
    'clanRank': 1,
    'previousClanRank': 1,
    'donations': 166,
    'donationsReceived': 0,
    'clanChestPoints': 0},
   {'tag': '#C22U88U2',
    'name': 'Bo$$',
    'role': 'coLeader',
    'lastSeen': '20250530T0045

In [105]:

# 1. Cargar el archivo correctamente
with open('clanes_data.json', 'r') as f:
    clan_data = json.load(f)  # clan_data es ahora un diccionario

# 2. Convertir a DataFrame (estructura deseada)
df = (
    pd.DataFrame.from_dict(clan_data, orient='index')
    
)

# 3. Si hay datos anidados (ej: "location": {"country": "Chile"}):
df_final = pd.json_normalize(
    df.to_dict('records'),  # Datos a normalizar
    sep='_',
    max_level= 3                # Separador para campos anidados
)

df_final['diccionario'] = df_final['memberList'].apply(lambda x: x[0])

dic_df = pd.json_normalize(df_final['diccionario'],  # Datos a normalizar
    sep='_',
    max_level= 3
)

df_final = pd.concat([df, dic_df], axis=1)

df_final.head(5)


'''''



# Mostrar resultado
df_final.head(5)

'''

"''\n\n\n\n# Mostrar resultado\ndf_final.head(5)\n\n"

In [106]:
df_final.head()

,tag,name,type,description,badgeId,clanScore,clanWarTrophies,location,requiredTrophies,donationsPerWeek,...,lastSeen,expLevel,trophies,clanRank,previousClanRank,donations,donationsReceived,clanChestPoints,arena_id,arena_name
#L9CRC0PG,#L9CRC0PG,Biscy Saucers,inviteOnly,1# Clan in Canada 🇨🇦 Better than the Rest! Ver...,16000076.0,90000.0,1650.0,"{'id': 57000047, 'name': 'Canada', 'isCountry'...",9000.0,2760.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
#GG0C0VR0,#GG0C0VR0,Jynxzi Verse,inviteOnly,,16000008.0,90000.0,180.0,"{'id': 57000006, 'name': 'International', 'isC...",9000.0,5024.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
#P00CUJ2R,#P00CUJ2R,Eternal Imperio,open,🔱Bienvenidos a la familia Eternal🔱 ⚜️Una nueva...,16000013.0,90000.0,2300.0,"{'id': 57000181, 'name': 'Panama', 'isCountry'...",9000.0,2126.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
#LC0GJGRV,#LC0GJGRV,los mercenarios,inviteOnly,"Se regalan 3 pass a los 3 top guerra. 2,200 pt...",16000077.0,90000.0,4922.0,"{'id': 57000169, 'name': 'Nicaragua', 'isCount...",9000.0,4895.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
#YVQV2VVY,#YVQV2VVY,BAD BOYS,inviteOnly,♨️CLAN COMPETITIVO♨️ATACAR EN GUERRA 4/4 ATAQU...,16000009.0,89460.0,3693.0,"{'id': 57000153, 'name': 'Mexico', 'isCountry'...",9000.0,5319.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [100]:
df_final['diccionario'] = df_final['memberList'].apply(lambda x: x[0])

# Paso 2: Convertir esa columna en columnas individuales
dic_df = pd.json_normalize(df_final['diccionario'])

# Paso 3: Unir al DataFrame original si quieres mantener otras columnas
#df_final = pd.concat([df, dic_df], axis=1).drop(columns=['datos', 'diccionario'])


In [74]:
tst = df[['tag','memberList']]
tst.head()

,tag,memberList
#L9CRC0PG,#L9CRC0PG,"[{'tag': '#JJU8V9YR', 'name': 'BigDanny224', '..."
#GG0C0VR0,#GG0C0VR0,"[{'tag': '#C0V0UQ9UY', 'name': 'Ryley', 'role'..."
#P00CUJ2R,#P00CUJ2R,"[{'tag': '#20Y8GVRLQ', 'name': 'david liu', 'r..."
#LC0GJGRV,#LC0GJGRV,"[{'tag': '#9PGU890Q9', 'name': 'MikeXGamer™', ..."
#YVQV2VVY,#YVQV2VVY,"[{'tag': '#JQ82089QR', 'name': 'xoguh12', 'rol..."


In [78]:
tst['memberList'][0][0]

/tmp/ipykernel_15491/384682349.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  tst['memberList'][0][0]


{'tag': '#JJU8V9YR',
 'name': 'BigDanny224',
 'role': 'coLeader',
 'lastSeen': '20250530T031133.000Z',
 'expLevel': 69,
 'trophies': 9000,
 'arena': {'id': 54000020, 'name': 'Valkalla'},
 'clanRank': 1,
 'previousClanRank': 1,
 'donations': 166,
 'donationsReceived': 0,
 'clanChestPoints': 0}

In [81]:
tst['diccionario'] = tst['memberList'].apply(lambda x: x[0])
tst.head()

/tmp/ipykernel_15491/1423004643.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tst['diccionario'] = tst['memberList'].apply(lambda x: x[0])


,tag,memberList,diccionario
#L9CRC0PG,#L9CRC0PG,"[{'tag': '#JJU8V9YR', 'name': 'BigDanny224', '...","{'tag': '#JJU8V9YR', 'name': 'BigDanny224', 'r..."
#GG0C0VR0,#GG0C0VR0,"[{'tag': '#C0V0UQ9UY', 'name': 'Ryley', 'role'...","{'tag': '#C0V0UQ9UY', 'name': 'Ryley', 'role':..."
#P00CUJ2R,#P00CUJ2R,"[{'tag': '#20Y8GVRLQ', 'name': 'david liu', 'r...","{'tag': '#20Y8GVRLQ', 'name': 'david liu', 'ro..."
#LC0GJGRV,#LC0GJGRV,"[{'tag': '#9PGU890Q9', 'name': 'MikeXGamer™', ...","{'tag': '#9PGU890Q9', 'name': 'MikeXGamer™', '..."
#YVQV2VVY,#YVQV2VVY,"[{'tag': '#JQ82089QR', 'name': 'xoguh12', 'rol...","{'tag': '#JQ82089QR', 'name': 'xoguh12', 'role..."


In [ ]:
i[0]

In [66]:
for i in df['memberList']:
    print(i.ittems())  # Imprime el nombre del primer miembro de cada clan

AttributeError: 'list' object has no attribute 'ittems'

In [ ]:
#transformando el dataframe
df_transformed = df.reset_index().rename(columns={'index': 'tag_clan'})
#normalizando columnas
df_final = json_normalize(df_transformed.to_dict('records'))

In [ ]:
df_final.info()

In [ ]:
df_final.sample(5)

In [ ]:
df_final.info()

In [ ]:
with open("token.txt", "r") as file:
    token = file.read().strip()
    print(f"Token usado: '{token}'") 

In [ ]:
import requests
import json
from time import sleep, time

# Leer el token
with open("token.txt", "r") as file:
    token = file.read().strip()

print(f"Token usado: '{token}'")  # Debug: verifica el token

base_url = "https://api.clashroyale.com/v1"
headers = {
    "Authorization": f"Bearer {token}",
    "Accept": "application/json"
}

# Test endpoint público (para verificar token)
def test_token():
    test_url = base_url + "/cards"
    try:
        response = requests.get(test_url, headers=headers)
        print(f"Test response: {response.status_code}")
        if response.status_code == 200:
            print("¡Token válido!")
        else:
            print(f"Error: {response.json()}")
    except Exception as e:
        print(f"Error en test: {e}")

test_token()  # Ejecuta esta prueba primero

# Si el test falla, detén el programa aquí
if input("¿El token es válido? (s/n): ").lower() != 's':
    exit()